In [9]:
import tensorflow as tf
import numpy as np
# 查询系统可用的 GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# 确保有可用的 GPU 如果没有, 则会报错
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# 设置参数,该段务必在运行jupyter的第一段代码执行，否则会无法初始化成功
# 仅在需要时申请显存空间（程序初始运行时消耗很少的显存，随着程序的运行而动态申请显存）
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### 一、自定义层

- 使用的主要数据结构是layer
- 实现自定义层的最佳方法是扩展tf.keras.layers.layer 类并实现
1. __init__ ： 可以在其中进行所有与输入无关的初始化，定义相关的层
2. build: 知道输入张量的形状并可以进行其余的初始化
3. call:在这里进行前向传播
注意：不一定需要在build中创建变量时，也可以在__init__中创建他们。

#### tf.keras.Model 和 tf.keras.layers.layer 区别和联系

- 通过继承 tf.keras.Model 编写自己的模型类
- 通过继承tf.keras.layers.layer 编写自己的层
- tf.keras中的模型和层都是继承tf.Module实现的
- tf.keras.Model继承tf.keras.layers.layer实现的

#### 自定义一个线性回归模型

In [4]:
from sklearn import datasets
iris = datasets.load_iris()
data =iris.data
target = iris.target

In [12]:
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
data[0:2] # 数据本身就算array，所以输入是完全可以直接被神经网络接收的

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2]])

In [37]:
data.shape

(150, 4)

In [38]:
target.shape

(150,)

#### 自定义线性模型的方法1 --- 最基础的方法

In [10]:
class Linear(tf.keras.layers.Layer):
    
    def __init__(self,units =1, input_dim = 4):
        super(Linear,self).__init__()
        w_init = tf.random_normal_initializer() # 随机初始化
        self.w = tf.Variable(initial_value = w_init(shape = (input_dim,units),dtype = 'float32'), trainable = True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value = b_init(shape = (units,), dtype = 'float32'),trainable = True)
    
    def call(self, inputs):
        return tf.matmul(inputs,self.w) + self.b

x = tf.constant(data)
linear_layer = Linear(units =1,input_dim = 4)
y = linear_layer(x)
print(y.shape)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


In [41]:
y[0:10]

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.32608113],
       [0.28710207],
       [0.2979414 ],
       [0.27652025],
       [0.32865408],
       [0.3403099 ],
       [0.29857594],
       [0.3103535 ],
       [0.26214537],
       [0.29026067]], dtype=float32)>

#### 自定义线性模型的方法2-- 使用self.add_weight创建变量

In [42]:
class Linear(tf.keras.layers.Layer):
    
    def __init__(self,units =1, input_dim = 4):
        super(Linear,self).__init__()
        self.w = self.add_weight(shape = (input_dim,units),initializer='random_normal',trainable = True)
        self.b = self.add_weight(shape=(units,),initializer='zeros',trainable = True)
        
    def call(self, inputs):
        return tf.matmul(inputs,self.w) + self.b
    
x = tf.constant(data)
linear_layer = Linear(units =1,input_dim = 4)
y = linear_layer(x)
print(y.shape)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


In [43]:
y[0:10]

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[-0.08645435],
       [-0.12379789],
       [-0.08351722],
       [-0.10863582],
       [-0.0715258 ],
       [-0.10008639],
       [-0.07180031],
       [-0.10114945],
       [-0.10675558],
       [-0.12223301]], dtype=float32)>

#### 自定义线性模型的方法3 --- build函数中创建变量

In [44]:
class Linear(tf.keras.layers.Layer):

    def __init__(self,units =32):
        super(Linear,self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape = (input_shape[-1],self.units),
        initializer = 'random_normal',
        trainable = True)
        self.b = self.add_weight(shape = (self.units,),
        initializer = 'random_normal',
        trainable = True)
        super(Linear,self).build(input_shape)

    def call(self,inputs):
        return tf.matmul(inputs,self.w) + self.b

In [45]:
x = tf.constant(data)
linear_layer = Linear(units = 1)
y = linear_layer(x)
print(y.shape)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(150, 1)


In [46]:
print('weight:',linear_layer.weights)
print('non-trainable weight:',linear_layer.non_trainable_weights) # 不参与训练的张量
print('trainable weight:',linear_layer.trainable_weights) # 参与训练的张量

weight: [<tf.Variable 'linear_5/Variable:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.09345417],
       [-0.03406332],
       [ 0.01242764],
       [-0.00207755]], dtype=float32)>, <tf.Variable 'linear_5/Variable:0' shape=(1,) dtype=float32, numpy=array([0.03450685], dtype=float32)>]
non-trainable weight: []
trainable weight: [<tf.Variable 'linear_5/Variable:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.09345417],
       [-0.03406332],
       [ 0.01242764],
       [-0.00207755]], dtype=float32)>, <tf.Variable 'linear_5/Variable:0' shape=(1,) dtype=float32, numpy=array([0.03450685], dtype=float32)>]


## 自定义层的注意事项

一、需要注意在自定义网络层时要重写get_config,示例如下

In [47]:
class MyDense(tf.keras.layers.Layer):

    def __init__(self,units =32, **kwargs):
        self.units = units
        super(MyDense,self).__init__(**kwargs)


     # build方法一般定义Layer需要被训练的参数   
    def build(self, input_shape):

        self.w = self.add_weight(shape = (input_shape[-1],self.units),
        initializer = 'random_normal',
        trainable = True,
        name = 'w')

        self.b = self.add_weight(shape = (self.units,),
        initializer = 'random_normal',
        trainable = True,
        name = 'b')

        super(MyDense,self).build(input_shape) # 相当于设置self.build = True

    # call方法一般定义正向传播运算逻辑，__call__方法调用了它
    def call(self,inputs):
        return tf.matmul(inputs,self.w) + self.b

    # 如果要让自定义的Layer通过Funcitional API 组合成模型时可以序列化，需要自定义get_config方法
    def get_config(self):
        config = super(MyDense,self).get_config()
        config.update({'units':self.units})
        return config

In [48]:
from sklearn import datasets
iris = datasets.load_iris()
data =iris.data
labels = iris.target

In [49]:
# 定义一个网络 函数式编程定义的网络
inputs = tf.keras.Input(shape=(4,))
x = MyDense(units = 16)(inputs) # 设置含有16个神经元的units中间层
x = tf.nn.tanh(x) # 使用tanh函数对神经元做转换
x = MyDense(units = 3)(x) # 定义输入的神经元的个数
predictions = tf.nn.softmax(x) # 使用softmax做转换
model = tf.keras.Model(inputs = inputs,outputs = predictions) # 通过一个方法对输入和输出联系起来

![鸢尾花数据集演示简单的神经网络结构](/home/hp/git/learningzone/huangpei/eat_tensorflow2_in_30_days/markdown_pics/鸢尾花数据集示意简单神经网络.png)

In [50]:
import numpy as np

In [51]:
data = np.concatenate((data,labels.reshape(150,1)),axis = -1)
np.random.shuffle(data) # 对数据做随机初始化，然后打乱数据的顺序

In [52]:
data.shape

(150, 5)

In [53]:
labels = data[:,-1] # 取最后一列数据
data = data[:,:4]

In [54]:
labels.shape

(150,)

In [55]:
data.shape

(150, 4)

In [56]:
# 定义优化器
# 定义损失函数 交叉熵损失函数
# 定义评估函数 acc

model.compile(optimizer=tf.keras.optimizers.Adam(),
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(data,labels, batch_size = 32, epochs = 100, shuffle = True)

Epoch 1/100
5/5 [==============================] - 0s 2ms/step - loss: 1.0975 - sparse_categorical_accuracy: 0.3333
Epoch 2/100
5/5 [==============================] - 0s 2ms/step - loss: 1.0954 - sparse_categorical_accuracy: 0.3333
Epoch 3/100
5/5 [==============================] - 0s 2ms/step - loss: 1.0935 - sparse_categorical_accuracy: 0.3333
Epoch 4/100
5/5 [==============================] - 0s 2ms/step - loss: 1.0916 - sparse_categorical_accuracy: 0.3333
Epoch 5/100
5/5 [==============================] - 0s 2ms/step - loss: 1.0896 - sparse_categorical_accuracy: 0.3333
Epoch 6/100
5/5 [==============================] - 0s 2ms/step - loss: 1.0871 - sparse_categorical_accuracy: 0.3333
Epoch 7/100
5/5 [==============================] - 0s 2ms/step - loss: 1.0850 - sparse_categorical_accuracy: 0.3333
Epoch 8/100
5/5 [==============================] - 0s 2ms/step - loss: 1.0816 - sparse_categorical_accuracy: 0.3333
Epoch 9/100
5/5 [==============================] - 0s 2ms/step - loss: 1

In [57]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
my_dense_4 (MyDense)         (None, 16)                80        
_________________________________________________________________
tf_op_layer_Tanh_2 (TensorFl [(None, 16)]              0         
_________________________________________________________________
my_dense_5 (MyDense)         (None, 3)                 51        
_________________________________________________________________
tf_op_layer_Softmax_2 (Tenso [(None, 3)]               0         
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


### 特别注意，如果前面不定义get_config，那么就没办法获取参数用于保存

In [59]:
model.save('keras_model_tf_version.h5')

二、若模型保存model.save报错如下图，则可能是自定义层的build中创建初始矩阵的时候，name属性没定义，导致报错

##### 错误类型
RuntimeError:Unable to create link (name already exists)

三、当自定义网络层并且有效保存模型后，希望使用tf.keras.models.load_model进行模型加载时，可能报错如下：

##### 错误类型
ValueError:Unknown layer:MyDense

解决方法：建立一个字典，键是自定义网络层时设定该层的名字，值是该自定义网络层的类名，该字典用于加载模型的时候使用，然后在tf.keras.models.load_model内传入custom_objects告知如何解析重建自定义网络层

In [63]:
# 重新加载如下
_custom_objects = {'MyDense':MyDense}

new_model = tf.keras.models.load_model('keras_model_tf_version.h5',custom_objects = _custom_objects)

In [64]:
y_pred = new_model.predict(data)

In [65]:
np.argmax(y_pred,axis = 1)

array([0, 2, 0, 0, 2, 2, 1, 2, 0, 0, 2, 2, 2, 2, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 2, 0, 1, 2, 1, 0, 1, 2, 1, 1, 0, 0, 2, 2, 2, 0, 2, 1, 0, 2,
       1, 0, 0, 1, 1, 2, 2, 0, 0, 1, 2, 0, 2, 2, 2, 1, 1, 2, 2, 1, 0, 0,
       2, 0, 2, 1, 0, 0, 2, 2, 2, 1, 0, 0, 1, 2, 1, 1, 1, 0, 0, 2, 1, 1,
       1, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 1, 2, 2, 2, 1, 1, 0, 0, 2,
       2, 0, 0, 2, 0, 0, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 2,
       2, 1, 0, 1, 1, 1, 1, 1, 0, 0, 2, 0, 2, 1, 2, 1, 1, 1])

In [66]:
labels

array([0., 2., 0., 0., 2., 2., 1., 2., 0., 0., 2., 2., 2., 2., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 1., 2., 0., 1., 2., 1., 0., 1., 1., 1., 1.,
       0., 0., 2., 2., 2., 0., 2., 1., 0., 2., 1., 0., 0., 1., 1., 2., 2.,
       0., 0., 1., 2., 0., 1., 2., 2., 1., 1., 2., 2., 1., 0., 0., 2., 0.,
       2., 1., 0., 0., 2., 2., 2., 1., 0., 0., 1., 2., 1., 1., 1., 0., 0.,
       2., 1., 1., 1., 0., 0., 0., 2., 2., 0., 2., 0., 2., 0., 2., 2., 1.,
       2., 1., 2., 1., 1., 0., 0., 1., 2., 0., 0., 2., 0., 0., 2., 1., 2.,
       2., 2., 2., 1., 2., 1., 1., 1., 0., 1., 1., 0., 2., 2., 1., 0., 1.,
       1., 1., 1., 1., 0., 0., 2., 0., 2., 1., 2., 1., 1., 1.])

四、当自定义一个网络层其名字和默认的tf.keras网络层一样的时候，可能会报错

五、当实现自定义网络层的时候，最好统一在初始化的时候传入可变参数**kwargs,这是因为在model推理的时候，有时我们需要对所有构成该模型的网络层进行统一的传参